# Trabalho prático de dados, parte 2

<u>__Descrição__</u>: Nesta etapa do trabalho prático, o grupo precisava aplicar algum algoritmo de aprendizagem
(regras de associação, regressão, aprendizado supervisionado ou aprendizado não-supervisionado)
para classificar ou agrupar os dados e, assim, tentar prever algum acontecimento desconhecido, com
foco em evasão, nota do ENEM e CRA.

## Imports básicos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

## Lendo dataset

In [ ]:
df = pd.read_csv("../Datasets/dadosFiltrados.csv")
df.info()

### Selecionando atributos significativos

Vamos primeiramente realizar uma plotagem dos atributos do datasets para ver se conseguimos encontrar alguma correlação pelos gráficos. Para isso, foram selecionadas algumas colunas que possivelmente apresentarão resultados significativos

Como o pairplot plota valores numéricos, foi necessário converter os valores `string` para categóricos, tornando possível a plotagem dos valores

In [ ]:
dfCat = df[[  # Selecionando as colunas relevantes para analise
    "Ano_Nascimento" ,
    "Sexo" ,
    "Campus" ,
    "Curso_Identificador" ,
    "UF_Nascimento" ,
    "Codigo_Situacao_Aluno" ,
    "Modalidade_Inscrita" ,
    "ENEM" ,
    "Num_Reprovacoes" ,
    "Raca" ,
    "CRA" ,
    "Area" ,
    "Admissao_Ano",
    "Saida_Ano",
]]
# Colunas não usada. As colunas abaixo foram desconsideradas ou por serem muito especificas ou por serem redundantes

# ["Identificador" , 
# "Curso" ,
# "UF_Nascimento" ,
# "Admissao" ,
# "Saida" ,
# "Situacao_Aluno" ,
# "Situacao_Aluno_Agrupada" ,
# "Admissao_Semestre",
# "Saida_Semestre"]

# Vamos converter os atributos desse dataset que são categoricos para variaveis discretas
colunas = dfCat.select_dtypes(include=['object']).columns
for col in colunas:
    dfCat[col] = dfCat[col].astype('category')
for col in colunas:
    dfCat[col] = dfCat[col].cat.codes

# Como dito na analise exploratoria, vamos remover os alunos que não se formaram ainda pois eles atrapalham a visualização
dfCat = dfCat[dfCat['Saida_Ano'] != 0]
dfCat


In [ ]:
# ## Plotando o pairplot

# plt.figure(figsize=(17, 17))
# sns.pairplot(dfCat, corner=True)
# plt.savefig('Imagens/pairplot.png', format='png', dpi=300)
# plt.close() # impede a impressão da figura na célula. Optamos por salvar a imagem e carregá-la no markdown

![Imagens/pairplot.png](Imagens/pairplot.png)

### Análise do pairplot

O pairplot nos permitiu visualizar alguns padrões muito interessantes, apesar do seu tamanho e complexiddade. Justamente por isso ele foi exportado dentro do arquivo [Imagens/pairplot.png](Imagens/pairplot.png), para permitir sua visualização de forma mais detalhada. 

Para exemplificar, melhor essas relações, vamos fazer também um heatmap dos valores para medir as correlações


In [ ]:
correlation = dfCat.corr()
matrix = np.triu(dfCat.corr())
plt.figure(figsize=(15,8))
sns.heatmap(correlation, xticklabels=correlation.columns, yticklabels=correlation.columns, annot=True, center=0, cmap='coolwarm', mask=matrix)

Note que o Heatmap, em paralelo ao pairplot, são muito esclarecedores em diversos aspectos. 

É fácil perceber que as variáveis de tempo, tais como ano de saída, ano de admissão, e ano de nascimento estão fortemente interligadas. Isso pode ser facilmente esclarecido por meio do senso comum: Boa parte dos alunos ingressantes na faculdade tendem a entrar alguns anos depois de terminarem o ensino médio. O mesmo vale para a saída da faculdade, que costuma ser o tempo necessário para concluir o curso, levandoo em conta uma certa margem de erro.

Outras relações que podem ser percebidas visualmente é em relação a reprovações e sexo. Apesar do heatmap não medir essa relação muito bem, o pairplot nos mostra que existem muito poucas mulheres com alto número de reprovações (veja no paiplot que a coluna de reprovações femininas é muito menor que a masculina). Com base em tudo isso, iremos realizar algumas análises preditivas.

## Previsão do tempo de duração do curso

Com base nos resultados anteriores, vamos tentar prever a duração de tempo do curso com base em atributos julgados relevantes. 

Após o tratamento de dados, o dataset foi enriquecido com novas colunas. Porém para determinadas análises, certo atributos acabam sendo redundantes. Além disso, outros atributos, tais como "Indentificador" não contribuem em nada para determinação de padrões. Por conta disso, eles também foram removidos. 

Por fim, também foram desconsiderados atributos com alto grau de complexidade e de variação, tais como a cidade de nascimento e o curso dos alunos. Esses atributos, por possuirem diversos valores possíveis, acabariam por deixar a análise muito complexa


In [ ]:
dfSaida = df.dropna()
dfSaida = dfSaida[dfSaida['Saida_Ano'] != 0]
dfSaida['Duracao'] = dfSaida['Saida_Ano'] - dfSaida['Admissao_Ano']
dfSaida = dfSaida[[
    "Sexo" ,
    "Campus" ,
    "Codigo_Situacao_Aluno" ,
    "Modalidade_Inscrita" ,
    "ENEM" ,
    "Num_Reprovacoes" ,
    "Raca" ,
    "CRA" ,
    "Area" ,
    "Duracao"]]

# Atributos desconsiderados
# ["Identificador" ,
# "Ano_Nascimento" ,
# "Curso" ,
# "UF_Nascimento" ,
# "Admissao" ,
# "Saida" ,
# "Situacao_Aluno" ,
# "Situacao_Aluno_Agrupada" ,
# "Curso_Identificador" ,
# "Admissao_Semestre",
# "Saida_Semestre"]

dfSaida.info()
df['Situacao_Aluno'].unique()

Na etapa acima, além da filtragem de atributos relevantes, também foi criada a coluna `Duracao`, que contém o tempo que o aluno permaneceu na faculdade.


#### Regressão com estratégia de One-Hot encoding

Para essa questão, a estratégia adotada foi a de regressão linear. Sabe-se que numa regressão, cada coluna recebe um peso (ou coeficiente) que é utilizado para calibrar a regressão. Porém os atributos categóricos acabam se tornando um problema para a regressão linear, visto que não é possível realizar uma multiplicação com um objeto ou outra variável que não seja numérica. Associar cada rótulo a uma variável discreta também seria errado, visto que uma variável aleatóriamente que recebeu um rótulo maior não seria necessáriamente mais importante (não teria uma força maior).

Para contornar esse problema, foi utilizado uma estratégia extremamente interessante conhecida como <u>__One Hot Encoding__</u>. A ideia é, para cada variável distinta de um determinado atributo, criar uma coluna que identifica se aquela variável está ou não presente naquele dado. 

Para exemplificar, imagine um dataset com uma coluna "Cor" onde os dados são classificados como "Azul", "Rosa" e "Preto". Após o OneHot, teríamos três colunas: "Cor_Azul", "Cor_Rosa" e "Cor_Preto". Um dado que originalmente era classificado como rosa terá agora valores `0,1,0` para as três colunas, respectivamente. 

In [ ]:
dfOneHot = pd.get_dummies(dfSaida, columns=[
    'Sexo', 
    'Campus',
    'Codigo_Situacao_Aluno', 
    'Modalidade_Inscrita', 
    'Raca',
    'Area',
    ], drop_first=True)
x = dfOneHot.drop(columns=['Duracao'])
y = dfOneHot['Duracao']
display(dfOneHot)
x.columns

Uma vez que os dados estão separados devidamente, é possível utilizar agora a regressão, atribuindo um peso diferente para cada coluna

In [ ]:

lm = LinearRegression()
lm.fit(x, y)

print('Coeficiente estimado: ', lm.coef_) # Coeficientes encontrados, ou seja, qual o peso de cada atributo
print(f"\nR2 (score): {lm.score(x,y)} (Ou {(lm.score(x,y)*100):.2f}%)") # R2 Score, ou seja, quão bom é o modelo

coeff_df = pd.DataFrame(lm.coef_, x.columns, columns=['Coefficient'])
coeff_df.sort_values(by='Coefficient', ascending=False) # Atributos ordenados por ordem de importancia

Veja que o nosso modelo teve um desemenho considerado muito bom (O valor de $R^2$ normalmente varia entre 0 e 1, sendo que em 1 ele consegue prever todos os dados com 100% de acurácia. No nosso caso, um score de 72.42% pode ser considerado um bom score). 

Quanto ao método utilizado, ele se mostrou como uma estratégia muito útil e interessante de abordagem de resolução de problemas. Além disso, esse modelo nos permite verificar um resultado muito interessante, mas que havia passado despercebido anteriormente: O atributo que mais impacta no tempo de duração do curso dos estudantes é o __Código de Situação do Aluno: Q__. Consultando novamente o [README](../README.md), o código de situação Q significa uma __"Matrícula condicional (desligamento reconsiderado)"__. 

Possivelmente esse alunos cursaram seus cursos por um período (ou talvez não chegaram a começar a cursar), foram desligados da UFV, porém posteriormente conseguiram recorrer e voltar ao curso, o que causou um tempo maior pra finalização do mesmo. Porém não é possível afirmar com certeza do que se trata o código devido a ausência de informações que o expliquem. 

Além desse atributo, diversos outros atributos interferem na duração final do curso. Confira na tabela. É interessante notar que alguns atributos que a priori pareciam os mais relevantes (tais como `num_reprovacoes`) ficaram atrás de outros que não pareciam tão relevantes.

Pode-se concluir que a geração do modelo foi um sucesso e que o modelo se mostrou muito interpretativo, ou seja, fácil de entender seus resultados e o porque de terem sido daquela forma

Veja abaixo um pequeno exemplo do modelo em funcionamento. Perceba que ele tende a dar resultados que são realmente próximos ao resultado real

In [ ]:
# Usando o modelo para prever alguns dados
teste = dfOneHot[:10]
teste['Predicted'] = lm.predict(teste.drop(columns=['Duracao']))
teste[['Duracao', 'Predicted']]


analises possiveis:
- mulheres reprovam menos
- reprovação por campus
- enem vs reprovçoes
- cra reprovaçoes